## STEP 1: Import required packages and define required functions
 - Import psycopg2
 - Import plotly
 - Define functions
     - to create db connection
     - to create a table
     - to execute an sql statement
     - to read the dataset file


In [2]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 1.2 MB/s eta 0:00:00


In [27]:
import psycopg2
from psycopg2 import Error
import plotly.express as px
import plotly.graph_objects as go

def create_new_database(host, dbname, user, password):
    try:
        # Connect to the default 'postgres' database or any existing database.
        connection = psycopg2.connect(
            host=host,
            dbname='postgres',
            user=user,
            password=password
        )
        connection.autocommit = True
        cursor = connection.cursor()

        # SQL command to create a new database.
        create_db_query = f"CREATE DATABASE {dbname};"

        # Execute the SQL command to create the new database.
        cursor.execute(create_db_query)

        print(f"Database '{dbname}' created successfully!")

    except Exception as e:
        print("Error:", e)

    finally:
        # Close the connection.
        if connection:
            cursor.close()
            connection.close()


def create_connection(host, database, user, password):
    try:
        # Establish a connection to the PostgreSQL database
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )

        # Return the connection object
        return conn

    except psycopg2.Error as e:
        print("Error: Unable to connect to the PostgreSQL database.")
        print(e)
        return None

def create_table(conn, create_table_sql, drop_table_name=None):
    try:
        cur = conn.cursor()
        if drop_table_name:
            cur.execute(f"DROP TABLE IF EXISTS {drop_table_name}")

        cur.execute(create_table_sql)
        conn.commit()
        print("Table created successfully.")

    except Error as e:
        print("Error: Unable to create table.")
        print(e)

def execute_sql_statement(sql_statement, conn):
    try:
        cur = conn.cursor()
        cur.execute(sql_statement)
        rows = cur.fetchall()
        return rows

    except Error as e:
        print("Error: Unable to execute SQL statement.")
        print(e)


def read_file(file_name):
    header = None
    row_values = []
    with open(file_name) as file:
        for line in file:
            if not line.strip():
                continue
            if not header:
                header = line.split('\t')
                continue
            data = line.split("'")
            row_values.append(data)
    return row_values

## STEP 2: Read the data and import into a database

In [28]:
#####################################################################
# Select required data from the txt file
#####################################################################
project_data = read_file("Police_Department_Incident_Reports__2018_to_Present.txt")
data=[]
for line in project_data:
    row = line[0].split("\t")
    data.append(row)

incident_id = []
incident_number = []
incident_month = []
incident_time = []
incident_year = []
incident_dow = []
incident_category = []
incident_subcategory = []
resolution = []
police_district = []
latitude = []
longitude = []
report_method = []

for i in data:

    incident_id.append(i[6])
    incident_number.append(i[7])
    incident_month.append(i[1].split("/")[0])
    incident_time.append(int(i[2].split(":")[0]))
    incident_year.append(i[3])
    incident_dow.append(i[4])
    incident_category.append(i[12])
    incident_subcategory.append(i[13])
    resolution.append(i[15])
    police_district.append(i[16])
    latitude.append(i[17])
    longitude.append(i[18])
    report_method.append(i[10])

incident_details = list(zip(incident_id,incident_number,incident_month,incident_year,incident_time,incident_dow,
                            incident_category,incident_subcategory,resolution,police_district,latitude,
                                                        longitude,report_method))

for i, row in enumerate(incident_details):
    incident_details[i] = [None if cell == '' else cell for cell in row]


In [29]:
#####################################################################
# Import data into a database
#####################################################################
host = "127.0.0.1"
database = "crimes_database"
user = "postgres"
password = "1209"
create_new_database(host=host, dbname=database, user=user, password=password)
conn = create_connection(host, database, user, password)
create_crime_summary_table_sql = '''CREATE TABLE IF NOT EXISTS Incident_Details (
    IncidentID INTEGER NOT NULL,
    IncidentNumber INTEGER NOT NULL,
    IncidentMonth INTEGER NOT NULL,
    IncidentYear INTEGER NOT NULL,
    IncidentTime INTEGER,
    IncidentDOW TEXT,
    IncidentCategory TEXT,
    IncidentSubcategory TEXT,
    Resolution TEXT,
    PoliceDistrict TEXT,
    Latitude REAL,
    Longitude REAL,
    ReportMethod TEXT,
    PRIMARY KEY (IncidentID,IncidentNumber))'''


with conn:
    cur = conn.cursor()
    create_table(conn, create_crime_summary_table_sql, drop_table_name=None)
    sql = ''' INSERT INTO Incident_Details(IncidentID, IncidentNumber, IncidentMonth,
         IncidentYear, IncidentTime, IncidentDOW, IncidentCategory, IncidentSubcategory,
         Resolution, PoliceDistrict, Latitude, Longitude, ReportMethod) 
         VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
         ON CONFLICT (IncidentID,IncidentNumber) DO NOTHING '''

    cur.executemany(sql, incident_details)

Error: database "crimes_database" already exists

Table created successfully.


## STEP 3: Data cleansing in the database table
 - Clean the data in below columns
     - FiledOnline
     - IncidentCategory
     - IncidentSubcategory
     - Latitude
     - Longitude

In [34]:
#####################################################################
# Cleaning the bad data in ReportMethod
#####################################################################
conn = create_connection(host, database, user, password)

with conn:
    report_method_sql1 = """UPDATE incident_details SET reportmethod = 'Online' WHERE reportmethod = 'TRUE' """
    report_method_sql2 = """UPDATE incident_details SET reportmethod = 'Offline' WHERE reportmethod IS NULL """
    cur = conn.cursor()
    cur.execute(report_method_sql1)
    cur.execute(report_method_sql2)
    conn.commit()

In [35]:
#####################################################################
# Cleaning the bad data in Incident_category and Incident_sub_category
#####################################################################
with conn:
    incident_cat_sql1 = """UPDATE Incident_Details SET IncidentCategory = 'Unidentified' WHERE IncidentCategory IS NULL """
    incident_cat_sql2 = """UPDATE Incident_Details SET IncidentCategory = 'Weapons Offense' WHERE IncidentCategory = 'Weapons Offence' """
    incident_cat_sql3 = """UPDATE Incident_Details SET IncidentCategory = 'Motor Vehicle Theft' WHERE IncidentCategory = 'Motor Vehicle Theft?' """
    incident_cat_sql4 = """UPDATE Incident_Details SET IncidentCategory = 'Other' WHERE IncidentCategory in ('Other Miscellaneous','Other Offenses') """
    incident_cat_sql5 = """UPDATE Incident_Details SET IncidentCategory = 'Suspicious Occ' WHERE IncidentCategory = 'Suspicious' """
    incident_sub_cat_sql1 = """UPDATE Incident_Details SET IncidentSubcategory = 'Unidentified' WHERE IncidentSubcategory IS NULL """
    incident_sub_cat_sql2 = """UPDATE Incident_Details SET IncidentSubcategory = 'Non-Criminal' WHERE IncidentCategory = 'Non-Criminal' """
    cur = conn.cursor()
    cur.execute(incident_cat_sql1)
    cur.execute(incident_cat_sql2)
    cur.execute(incident_cat_sql3)
    cur.execute(incident_cat_sql4)
    cur.execute(incident_cat_sql5)
    cur.execute(incident_sub_cat_sql1)
    cur.execute(incident_sub_cat_sql2)
    conn.commit()

In [36]:
#####################################################################
# Cleaning the bad data in Latitude and Longitude
#####################################################################
with conn:
    latitude_and_logitude_sql1 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Out of SF' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Out of SF' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Out of SF' and Latitude IS NULL """
    latitude_and_logitude_sql2 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Park' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Park' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Park' and Latitude IS NULL """
    latitude_and_logitude_sql3 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Richmond' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Richmond' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Richmond' and Latitude IS NULL """
    latitude_and_logitude_sql4 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Taraval' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Taraval' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Taraval' and Latitude IS NULL """
    latitude_and_logitude_sql5 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Ingleside' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Ingleside' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Ingleside' and Latitude IS NULL """
    latitude_and_logitude_sql6 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Bayview' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Bayview' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Bayview' and Latitude IS NULL """
    latitude_and_logitude_sql7 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Tenderloin' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Tenderloin' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Tenderloin' and Latitude IS NULL """
    latitude_and_logitude_sql8 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Southern' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Southern' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Southern' and Latitude IS NULL """
    latitude_and_logitude_sql9 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Mission' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Mission' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Mission' and Latitude IS NULL """
    latitude_and_logitude_sql10 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Northern' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Northern' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Northern' and Latitude IS NULL """
    latitude_and_logitude_sql11 = """UPDATE Incident_Details SET Latitude = (SELECT AVG(Latitude) 
    FROM Incident_Details WHERE PoliceDistrict = 'Central' and Latitude IS NOT NULL),
    Longitude = (SELECT AVG(Longitude) FROM Incident_Details WHERE PoliceDistrict = 'Central' and Longitude IS NOT NULL)
    WHERE PoliceDistrict = 'Central' and Latitude IS NULL """
    cur = conn.cursor()
    cur.execute(latitude_and_logitude_sql1)
    cur.execute(latitude_and_logitude_sql2)
    cur.execute(latitude_and_logitude_sql3)
    cur.execute(latitude_and_logitude_sql4)
    cur.execute(latitude_and_logitude_sql5)
    cur.execute(latitude_and_logitude_sql6)
    cur.execute(latitude_and_logitude_sql7)
    cur.execute(latitude_and_logitude_sql8)
    cur.execute(latitude_and_logitude_sql9)
    cur.execute(latitude_and_logitude_sql10)
    cur.execute(latitude_and_logitude_sql11)
    conn.commit()

## STEP 4: Derive insights from the data
 - Below are the insights we derived from the data
    - Insight 1: Monthly Crime Variation
    - Insight 2: Day of the Week Analysis
    - Insight 3: Time of Day Analysis
    - Insight 4: Filed Online vs Offline
    - Insight 5: Year-wise crime counts per district
    - Insight 6: Top 10 Crime Categories
    - Insight 7: Year-wise crime counts for different violent crimes


In [37]:
#####################################################################
# Insight 1: Monthly Crime Variation
#####################################################################
conn = create_connection(host, database, user, password)
sql1 = """SELECT IncidentMonth AS Month, COUNT(1)
FROM Incident_Details 
GROUP BY IncidentMonth"""

insight1 = execute_sql_statement(sql1,conn)

month = [row[0] for row in insight1]
crime_count = [row[1] for row in insight1]

trace = go.Scatter(x=month, y=crime_count, fill='tozeroy')
layout = go.Layout(title='Monthly Crime Variation',
                   xaxis={'title': 'Month'},
                   yaxis={'title': 'Number of Incidents'},
                   height=600)

fig = go.Figure(data=[trace], layout=layout)

fig.show()


In [38]:
#####################################################################
# Insight 2: Day of the Week Analysis
#####################################################################
sql2 = """SELECT IncidentDOW, COUNT(1) 
FROM Incident_Details 
GROUP BY IncidentDOW 
ORDER BY 1"""
insight2 = execute_sql_statement(sql2,conn)
labels = [row[0] for row in insight2]
values = [row[1] for row in insight2]

trace = go.Pie(labels=labels, values=values)
layout = go.Layout(title='Incidents by Day of Week')
fig = go.Figure(data=[trace], layout=layout)
fig.update_traces(hole=0.4, hoverinfo="label+percent+value", textinfo="label+percent", textfont_size=12,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [39]:
#####################################################################
# Insight 3: Time of Day Analysis
#####################################################################
sql3 = """SELECT IncidentTime, COUNT(1) 
FROM Incident_Details 
GROUP BY IncidentTime
ORDER BY 1 DESC"""
insight3 = execute_sql_statement(sql3,conn)
hour = [row[0] for row in insight3]
crime_count = [row[1] for row in insight3]

fig = px.bar(insight3, x=hour , y=crime_count)
fig.update_layout(
    xaxis_title='Hour of the Day',
    yaxis_title='Number of Incidents',
    title='Time of Day Analysis'
)
fig.show()

In [40]:
#####################################################################
# Insight 4: Filed Online vs Offline
#####################################################################
sql4 = """SELECT ReportMethod, COUNT(1)
FROM Incident_Details
GROUP BY ReportMethod"""
insight4 = execute_sql_statement(sql4,conn)

report_method = [row[0] for row in insight4]
counts = [row[1] for row in insight4]

fig = px.bar(y=report_method, x=counts, orientation='h', 
             title='Online Vs Offline Reports', 
             labels={'y':'Report Method', 'x':'Number of Incidents'})
fig.show()



In [41]:
#####################################################################
# Insight 5: Year-wise crime counts per district
#####################################################################
sql5 = """SELECT DISTINCT IncidentYear,PoliceDistrict, COUNT(1)
 FROM Incident_Details
 GROUP BY IncidentYear,PoliceDistrict 
 ORDER BY 2,1"""
insight5 = execute_sql_statement(sql5,conn) 
rows = [{'Year': row[0], 'Police District': row[1], 'Count': row[2]} for row in insight5]

fig = px.bar(rows, x='Year', y='Count', color='Police District',
             labels={'Year': 'Year', 'Count': 'Count', 'Police District': 'Police District'},
             title='Incidents by Year and Police District')
fig.show()


In [42]:
#####################################################################
# Insight 6: Top 10 Crime Categories
#####################################################################
sql6 = """SELECT IncidentCategory, COUNT(1)
FROM Incident_Details
GROUP BY IncidentCategory
ORDER BY 2 DESC
LIMIT 10"""
insight6 = execute_sql_statement(sql6,conn)

categories = [row[0] for row in insight6]
counts = [row[1] for row in insight6]

trace = go.Pie(labels=categories, values=counts)
layout = go.Layout(title='Top 10 Crime Categories')
fig = go.Figure(data=[trace], layout=layout)
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.2)))
fig.show()



In [43]:
#####################################################################
# Insight 7: Year-wise crime counts for different violent crimes
#####################################################################
sql7 = """SELECT DISTINCT IncidentYear, IncidentCategory, COUNT(2)
FROM Incident_Details
WHERE IncidentCategory IN ('Homicide', 'Rape','Assault' ,'Robbery' ,'Arson','Suicide','Weapons Offense')
GROUP BY IncidentYear, IncidentCategory
ORDER BY 3 DESC"""
insight7 = execute_sql_statement(sql7,conn)

years = [row[0] for row in insight7]
categories = [row[1] for row in insight7]
counts = [row[2] for row in insight7]

# Create a unique list of categories for x-axis labels
unique_categories = list(set(categories))

# Create a list of colors for each category
colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)']

data = []
for i, category in enumerate(unique_categories):
    category_counts = [count for j, count in enumerate(counts) if categories[j] == category]
    data.append(go.Bar(x=years, y=category_counts, name=category, marker_color=colors[i]))

layout = go.Layout(title='Frequency Distribution of Violent Incident Categories by Year',
                   xaxis={'title': 'Year'},
                   yaxis={'title': 'Count'})

fig = go.Figure(data=data, layout=layout)
fig.show()

## STEP 5: Time Series Analysis

In [48]:
#####################################################################
# Time Series Analysis of Crimes in San Franscisco
#####################################################################

import pandas as pd

conn = create_connection(host, database, user, password)
sql = '''
    SELECT IncidentYear || '-' || IncidentMonth AS Year_Month, COUNT(1)
    FROM Incident_Details 
    WHERE IncidentYear || '-' || IncidentMonth <= '2023-6'
    GROUP BY IncidentYear, IncidentMonth
'''

result = execute_sql_statement(sql, conn)

df = pd.DataFrame(result, columns=['Year_Month', 'Crime_Count'])

df['Year_Month'] = pd.to_datetime(df['Year_Month'])

df = df.sort_values('Year_Month')

df['Moving_Average'] = df['Crime_Count'].rolling(window=12, min_periods=1).mean()

fig = px.line(df, x='Year_Month', y=['Crime_Count', 'Moving_Average'])
fig.update_layout(
    xaxis_title='Year-Month',
    yaxis_title='Number of Incidents',
    title='Time Series of Crime Incidents in SFO'
)

fig.show()